In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as scy
from scipy import stats
from sklearn import decomposition
from scipy import interpolate
#%matplotlib qt5
#plt.rcParams["figure.figsize"] = (7,7)
import itertools
from numpy import pi
import emcee
import corner

In [2]:
gflux = np.load("gridflux.npy")
gtemps = np.load("gridtemps.npy")
gwl = np.load("gridwl.npy")

In [3]:
numT = len(gtemps)

In [4]:
nparams = 1
modelNsamples = len(gwl)
Ngrid = numT 
M = Ngrid*nparams

In [5]:
step1 = (0.1*gflux[3,:] + 0.9*gflux[4,:])
samplesDesired = 200
stepsize = int(modelNsamples/samplesDesired)
step2 = np.zeros((int(np.ceil(modelNsamples/stepsize)),2))
counter = 0
while counter < modelNsamples:
    tempstep = min(stepsize,modelNsamples-counter)
    subsample = step1[counter:counter+tempstep]
    noise = np.random.normal(loc=0, scale=0.01)
    #print(int(counter/stepsize),counter, modelNsamples)
    step2[int(counter/stepsize),:] = [np.mean(gwl[counter:counter+tempstep]),\
                       np.mean(subsample+noise)]
    counter+=tempstep

In [6]:
# plt.plot(step2[:,0],step2[:,1],alpha=0.5)
# plt.plot(gwl,gflux[3,:],alpha=0.5)
# plt.plot(gwl,gflux[15,:],alpha=0.5)

# plt.show()

In [7]:
Nsamples = len(step2)

In [8]:
#vectors for the parameter space
xv = gtemps
wl = step2[:,0]  #need data for this

In [9]:
templateData = np.zeros((Ngrid,Nsamples))
for i in range(numT):
    tempDat = interpolate.interp1d(gwl,gflux[i,:],kind='cubic')(wl)
    templateData[i] = tempDat

In [10]:
def spectral_whitener(specgrid):
    ximu = np.mean(specgrid,axis=0)
    xisigma = np.std(specgrid,axis=0)
    return ximu, xisigma, (specgrid-ximu)/xisigma

def eigen_spec(whitened, s=0.98):
    pca = decomposition.PCA()
    pca.fit(whitened)
    comps = pca.explained_variance_ratio_
    psum = 0
    count = 0
    l = len(comps)
    while psum < s and count < l:
        psum+=comps[count]
        count+=1
    xik = [0]*count
    counter = 0
    l2 = len(whitened)
    global nparams
    while counter < count:
        xik[counter] = pca.components_[counter]*\
        pca.singular_values_[counter]/(l2*nparams)**(1/2)
        counter+=1
    return count, np.array(xik).T

def w_gen(whitened, phi):
    l = whitened.shape[0]
    wgrid = np.zeros((l, phi.shape[1]))
    reuse_mat = np.linalg.pinv(phi)
    for i in range(l):
        wgrid[i] = reuse_mat@whitened[i]
    return wgrid.T
 
def phi_grid_maker(phi):
    global M
    Im = np.eye(M)
    phigrid = np.kron(Im,phi[:,0])
    for i in range(1,phi.shape[1]):
        phigrid = np.concatenate((phigrid,np.kron(Im,phi[:,i])),axis=0)
    return phigrid.T

def klist_maker(xvec, ak_list, lk_list):
    alist = ak_list**2
    llist = lk_list**(-2)
    Ngrid = xvec.shape[0]
    combos = list(itertools.combinations_with_replacement(np.arange(Ngrid),2))
    m = len(alist)
    Rlist = [np.atleast_1d(xvec[i]-xvec[j]) for i,j in combos]
    klist = [0]*m
    for i in range(m):
        ele = np.zeros((Ngrid,Ngrid))
        for index, (row,col) in enumerate(combos):
            covar = alist[i]*\
            np.exp(-0.5*Rlist[index].T@np.diag(llist[i])@Rlist[index])
            
            ele[row,col] = ele[col,row] = covar
        klist[i] = ele
    return klist

def sigma_grid_maker(klist):
    return scy.linalg.block_diag(*klist)

def kstar_list_maker(xvec, thetaStar, ak_list, lk_list):
    alist = ak_list**2
    llist = lk_list**(-2)
    Ngrid = xvec.shape[0]
    m = len(alist)
    kstar_list=[0]*m
    Rlist = [np.atleast_1d(xvec[i]-thetaStar) for i in range(Ngrid)]
    reusedMats = [np.diag(llist[i]) for i in range(m)]
    for i in range(m):
        ele = np.zeros((Ngrid))
        for j,item in enumerate(Rlist):
            ele[j] = alist[i]*np.exp(-0.5*item.T@reusedMats[i]@item)
        kstar_list[i] = ele
    return kstar_list

def sigmastar_grid_maker(kstar_list):
    return scy.linalg.block_diag(*kstar_list).T

def thetaStar_func(thetaStar, xv, alist, llist, lambda_xi,wgrid,
                   phi_grid,sigma_grid,sigmastar):
    kstarlist = kstar_list_maker(xv, thetaStar, alist, llist)
    sigmastar_grid = sigmastar_grid_maker(kstarlist)
    Mm = phi_grid.shape[1]
    usefulmat = np.linalg.solve(lambda_xi*(phi_grid.T)@\
                                phi_grid +sigma_grid,np.eye(Mm))
    mu_w = sigmastar_grid.T@usefulmat@wgrid
    sigma_w = sigmastar - sigmastar_grid.T@usefulmat@sigmastar_grid
    return mu_w, sigma_w

# def C_maker(wl,aG,lG,r0_coeff=4):
#     l = wl.shape[0]
#     C = np.zeros((l,l))
#     r0 = r0_coeff*lG
#     #might as well calculate these ahead of looping
#     const1 = 1.5e5 #c/2, units have to jive with lG
#     const2 = pi/r0
#     const3 = (3)**(1/2)/lG
#     combos = itertools.combinations_with_replacement(np.arange(l),2)
#     for i, j in combos:
#         rij = const1*(wl[j]-wl[i])/(wl[i]+wl[j])#wl[j] > wl[i] these are already sorted
#         if rij > r0 :
#             continue
#         C[i,j] = C[j,i] = 0.5*(1+np.cos(const2*rij))*\
#         aG*(1+rij*const3)*np.exp(-rij*const3)
#     return C
def C_maker(wl,aG,lG,r0_coeff=4):
    l = wl.shape[0]
    C = aG*np.eye(l)
    r0 = r0_coeff*lG
    #might as well calculate these ahead of looping
    const1 = 1.5e5 #c/2, units have to jive with lG
    const2 = pi/r0
    const3 = (3)**(1/2)/lG
    i, j = 0,1
    while i < l:
        while j < l:
            rij = const1*(wl[j]-wl[i])/(wl[i]+wl[j])#wl[j] > wl[i] these are already sorted
            if rij > r0 :
                break
            C[i,j] = C[j,i] = 0.5*(1+np.cos(const2*rij))*\
            aG*(1+rij*const3)*np.exp(-rij*const3)
            j+=1
        i+=1
        j=i+1
    return C

def omnibus(thetaStar, xv, wl, aG, lG, alist, llist, lambda_xi, wgrid, phi,
            xi_mu,xi_sigma,phi_grid):
    C = C_maker(wl, aG, lG)
    klist = klist_maker(xv, alist, llist)
    sigma_grid = sigma_grid_maker(klist)
    sigmastar = scy.linalg.block_diag(*alist**2)
    mu_w, sigma_w = thetaStar_func(thetaStar, xv, alist, llist, lambda_xi, 
                                   wgrid, phi_grid,sigma_grid,sigmastar)
    wsample = stats.multivariate_normal(mu_w,sigma_w).rvs()
    intermediate_mat = np.diag(xi_sigma)@phi
    mu_combined = xi_mu + intermediate_mat@mu_w
    sigma_combined = intermediate_mat@sigma_w@(intermediate_mat.T) + C
    return mu_combined, sigma_combined


def arglistformatter(arglst,m,nparams):
    #put an assertion about arglist length
    thetStar = np.array(arglst[:nparams])
    aaG = arglst[nparams]
    llG = arglst[nparams+1]
    aklist = np.array(arglst[nparams+2:nparams+2+m])
    lklist = np.array(arglst[nparams+2+m:-1]).reshape((m,nparams))
    lambdaxi = arglst[-1]
    return thetStar, aaG, llG, aklist, lklist, lambdaxi

def lnlikelihoodParentFunc(fobs, xv, wgrid, phi,xi_mu,xi_sigma, phi_grid, 
                           a_xi, b_xi,aG_dist,lG_dist,lambda_xi_dist,
                           ak_dist,lk_dist,m,nparams):
    def lnprob(argslist):
        thetaStar, aG, lG, alist, llist, lambda_xi = \
        arglistformatter(argslist,m,nparams)
        
        mu, sigma = omnibus(thetaStar, xv, wl, aG, lG, alist, llist, 
                            lambda_xi, wgrid, phi,xi_mu,xi_sigma,phi_grid)
        p = stats.multivariate_normal(mu,sigma).pdf(fobs)*aG_dist.pdf(aG)*\
    lG_dist.pdf(lG)*lambda_xi_dist.pdf(lambda_xi)*\
    np.prod(ak_dist.pdf(alist))*np.prod(lk_dist.pdf(llist))
        if p <= 0:
            return -np.inf
        return np.log(p)
    return lnprob


In [11]:
xi_mu, xi_sigma, whiten = spectral_whitener(templateData)
m, phi = eigen_spec(whiten,s=0.98)
wvec = w_gen(whiten, phi)
wgrid = wvec.flatten()
phi_grid = phi_grid_maker(phi)

In [12]:
a_xi = 1
b_xi = 1e4
aG_loc = 3
aG_scale = 1
lG_loc = 10
lG_scale = 5
thetaStar_loc = xv[0]
thetaStar_scale = xv[-1]-xv[0]
ak_loc = 2
ak_scale = 1.2
lk_loc = 10
lk_scale = 5

In [13]:

aG_dist = stats.uniform(loc=aG_loc, scale = aG_scale)
lG_dist = stats.uniform(loc=lG_loc,scale=lG_scale)
lambda_xi_dist = stats.gamma(a_xi,scale=b_xi)
thetaStar_dist = stats.uniform(loc=thetaStar_loc,scale=thetaStar_scale)
ak_dist = stats.uniform(loc=ak_loc, scale=ak_scale)
lk_dist = stats.uniform(loc=lk_loc, scale=lk_scale)

#use these initial guesses to define the bounding ball

In [14]:
aG = aG_dist.rvs()
lG = lG_dist.rvs()
lambda_xi = lambda_xi_dist.rvs()
alist = ak_dist.rvs(size=m)
llist = lk_dist.rvs(size=(m,nparams))
thetaStar = thetaStar_dist.rvs(size=nparams)


In [15]:
mu, sigma = omnibus(thetaStar, xv, wl, aG, lG, alist, llist, 1,wgrid,phi, xi_mu,\
                    xi_sigma,phi_grid)
print(np.linalg.slogdet(sigma))#check to ensure the covariance isn't too spread out

(1.0, 224.2469734596805)


In [16]:

ndims = nparams + 3 + m*(1+nparams)
walkers = 8*ndims
cycles = 100

In [17]:
pos2 = [[*thetaStar_dist.rvs(size=nparams).flatten(),aG_dist.rvs(),lG_dist.rvs(),
           *ak_dist.rvs(size=m).flatten(),*lk_dist.rvs(size=(m,nparams)).flatten(),
         lambda_xi_dist.rvs()] for i in range(walkers)]

In [18]:
# def lnlikelihoodParentFunc(fobs, xv, wgrid, phi,xi_mu,xi_sigma, phi_grid, a_xi, b_xi,
#                           aG_dist,lG_dist,lambda_xi_dist,ak_dist,lk_dist,m,nparams):
#     def lnprob(argslist):
emceeFunc = lnlikelihoodParentFunc(step2[:,1], xv, wgrid, phi,xi_mu,xi_sigma,
                                   phi_grid,a_xi,b_xi,aG_dist,lG_dist,
                                   lambda_xi_dist,ak_dist,lk_dist,m,nparams)

In [19]:
lnposteriorlikelihood = np.zeros(walkers)

In [20]:
for i in range(walkers):
    lnposteriorlikelihood[i] = emceeFunc(pos2[i])

In [21]:

sampler = emcee.EnsembleSampler(walkers, ndims, emceeFunc)

In [22]:
import cProfile

# Set up profiling
pr = cProfile.Profile()
pr.enable()

# Actually run a command we want to time
storage = sampler.run_mcmc(pos2,cycles, lnprob0 = lnposteriorlikelihood)

# Done and print
pr.disable()
pr.print_stats(sort='time')


         62089092 function calls in 174.082 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    16000   46.973    0.003   48.691    0.003 decomp.py:240(eigh)
     8000   34.349    0.004   66.347    0.008 <ipython-input-10-805dc7ac4977>:41(klist_maker)
     8000   16.860    0.002   17.169    0.002 <ipython-input-10-805dc7ac4977>:105(C_maker)
  5576000   13.820    0.000   22.147    0.000 twodim_base.py:244(diag)
     8000   11.903    0.001   20.242    0.003 <ipython-input-10-805dc7ac4977>:79(thetaStar_func)
  7961990    5.147    0.000    8.366    0.000 numeric.py:484(asanyarray)
  8346291    4.100    0.000    4.100    0.000 {built-in method numpy.core.multiarray.array}
     8000    3.604    0.000    3.932    0.000 linalg.py:296(solve)
  5704002    3.558    0.000    3.558    0.000 {built-in method numpy.core.multiarray.zeros}
  2052646    3.140    0.000    9.431    0.000 shape_base.py:9(atleast_1d)
     8000    2.525    0.000 

In [23]:
samples = sampler.chain[:, 50:, :].reshape((-1, ndims))

In [24]:
fig = corner.corner(samples,labels=[r'T', 'aG','lG','ak1','ak2','ak3','lk1','lk2',
                                    'lk3',r'$\lambda_{\xi}$'],
                   quantiles=[0.16,0.5,0.84])
# plt.tight_layout()
# plt.show()
fig.savefig("ThirteenthRun.pdf")
plt.close()

In [25]:

xv[3]

4300.0

In [26]:
#parallel doesn't work right now
#sampler2 = emcee.EnsembleSampler(walkers, ndims, emceeFunc,threads=8)

In [27]:
#storage2 = sampler2.run_mcmc(pos2,cycles, lnprob0 = lnposteriorlikelihood)

In [28]:
'''
notes:
runs 1-3: 2*ndims walkers, 100 cycles
runs 4-6: 4*ndims walkers, 200 cycles
runs 7-9: 2*ndims walkers, 400 cycles
runs 10-13 : 8*ndims walkers, 100 cycles'''

'\nnotes:\nruns 1-3: 2*ndims walkers, 100 cycles\nruns 4-6: 4*ndims walkers, 200 cycles\nruns 7-9: 2*ndims walkers, 400 cycles\nruns 10-13 : 8*ndims walkers, 100 cycles'